<a href="https://colab.research.google.com/github/yuzgulfatih/Papcorns/blob/main/model_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "sd-legacy/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_image(prompt: str, negative_prompt: str):
    images = pipe(prompt, negative_prompt=negative_prompt).images
    return images[0]

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2rl0ul48lO2uHu73cRakPBYuL2P_2j8TUfWwJFtBJb9NtNEus")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from io import BytesIO

app = Flask(__name__)
CORS(app)  # This will allow all domains by default

@app.route('/generate-image', methods=['POST'])
def generate_image_route():
    try:
        data = request.get_json()
        prompt = data.get('prompt')
        negative_prompt = data.get('negative_prompt')

        if not prompt or not negative_prompt:
            return jsonify({'error': 'Prompt and negative prompt is required!'}), 400

        image = generate_image(prompt, negative_prompt)

        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

        return jsonify({'image': img_str})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Ngrok URL: {public_url}")

    app.run(host='0.0.0.0', port=5000)